In [1]:
import torch
import torch.nn as nn
import cv2
from model.resnet import ResNet50, ResNet101, ResNet152

model = ResNet101(num_classes=10, channels=1).cuda()
model.load_state_dict(torch.load('./Cady_model_99.pth'))
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (i_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, m

In [4]:
import os
from tool import *
from tqdm import tqdm
# img = cv2.imread('./frames/s03_cond3_2/frame_0322.jpg', cv2.IMREAD_GRAYSCALE)

path = './test_frames/s1_01'
output_path = './output/s1_01'
if not os.path.exists(output_path):
    os.makedirs(output_path)
for file in tqdm(os.listdir(path)):
    img = cv2.imread(os.path.join(path, file), cv2.IMREAD_GRAYSCALE)
    height_extension = 100
    new_height = img.shape[0] + height_extension
    new_image = np.zeros((new_height, img.shape[1]), dtype=np.float32)
    new_image[:img.shape[0], :] = img
    img = cv2.resize((new_image/255)*2 - 1, (512, 512))
    input_img = torch.tensor(img).float().cuda()
    input_img = input_img.unsqueeze(0).unsqueeze(0)
    output = model(input_img)
    output = output.squeeze().cpu().detach().numpy()
    result = draw_img(img, output)
    cv2.imwrite(os.path.join(output_path, file), result)

# height_extension = 100
# new_height = img.shape[0] + height_extension
# new_image = np.zeros((new_height, img.shape[1]), dtype=np.float32)
# new_image[:img.shape[0], :] = img
# img = cv2.resize((new_image/255)*2 - 1, (512, 512))
# input_img = torch.tensor(img).float().cuda()
# input_img = input_img.unsqueeze(0).unsqueeze(0)
# print(input_img.shape)

100%|██████████| 1999/1999 [00:22<00:00, 88.87it/s]


In [37]:
from tool import *
output = model(input_img)
print(output.shape)

output = output.squeeze().cpu().detach().numpy()
result = draw_img(img, output)
cv2.imwrite('result.png', result)

torch.Size([1, 10])
(600, 600)


True

## sequence input model

In [5]:
import os
import numpy as np
import torch
import torch.nn as nn
from vit_pytorch.vivit import ViT

seq_len = 15

model= ViT(
    image_size = 512,          # image size
    frames = seq_len,               # number of frames
    image_patch_size = 64,     # image patch size
    frame_patch_size = 3,      # frame patch size
    num_classes = 10,
    channels = 1,              # number of image channels
    dim = 1024,
    spatial_depth = 6,         # depth of the spatial transformer
    temporal_depth = 6,        # depth of the temporal transformer
    heads = 8,
    mlp_dim = 1024,
).cuda()

model.load_state_dict(torch.load('../output_models/Cady_model_seq_299.pth'))

<All keys matched successfully>

In [2]:

from pykalman import KalmanFilter

def apply_kalman_filter(keypoints):
    num_frames, num_coordinates = keypoints.shape
    num_keypoints = num_coordinates // 2
    filtered_keypoints = np.zeros_like(keypoints)

    for i in range(num_keypoints):
        # 提取第i个关键点的(x, y)坐标
        keypoint_data = keypoints[:, i*2:(i+1)*2]
        
        kf = KalmanFilter(
            initial_state_mean=keypoint_data[0],
            initial_state_covariance=np.eye(2),
            transition_matrices=np.eye(2),
            observation_matrices=np.eye(2),
            transition_covariance=0.01 * np.eye(2),
            observation_covariance=0.1 * np.eye(2)
        )
        
        # 过滤关键点
        filtered_state_means, _ = kf.filter(keypoint_data)
        filtered_keypoints[:, i*2:(i+1)*2] = filtered_state_means

    return filtered_keypoints

In [16]:
import os
from tool import *
from tqdm import tqdm
# img = cv2.imread('./frames/s03_cond3_2/frame_0322.jpg', cv2.IMREAD_GRAYSCALE)

# path = '../test_frames/s1_01'
# output_path = '../output/s1_01'

def draw_pred_img(path, model, seq_len=15):
    kp_array = np.zeros((len(os.listdir(path)), 10))
    for frame in tqdm(range(len(os.listdir(path)))):
        frames = []
        p = 1
        for i in range(seq_len):
            fi = i - seq_len // 2 + frame
            if fi < 0:
                img_path = f'{path}/frame_0000.jpg'
            else:
                img_path = f'{path}/frame_{fi:04d}.jpg'

            # print(img_path)
            if not os.path.exists(img_path):
                fi = fi - p
                p += 1
                img_path = f'{path}//frame_{fi:04d}.jpg'

            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            height_extension = 100
            new_height = img.shape[0] + height_extension
            new_image = np.zeros((new_height, img.shape[1]), dtype=np.float32)
            new_image[:img.shape[0], :] = img
            new_image = cv2.resize(new_image, (512, 512))
            new_image = (new_image / 255.0) * 2 - 1
            new_image = np.expand_dims(new_image, axis=0)
            new_image = torch.tensor(new_image, dtype=torch.float32)
            frames.append(new_image)
            # break

        frames = torch.stack(frames)
        frames = frames.permute(1, 0, 2, 3).unsqueeze(0).cuda()

        output = model(frames)
        output = output.squeeze().cpu().detach().numpy()
        # print(output.shape)
        kp_array[frame] = output
        
    return kp_array
    # print(frame)
    # print(frames[0][:,seq_len//2,:,:].shape, output.shape)
    # result = draw_img(torch.squeeze(frames[0][:,seq_len//2,:,:]).cpu().detach().numpy(), output)
    # cv2.imwrite(os.path.join(output_path, f'frame_{frame:04d}.jpg'), result)
    
    # print(frames.shape, frame)
def draw_smth_kp(path, output_path, smooth_kp, json_dict):
    for frame in tqdm(range(len(os.listdir(path)))):
        img_path = f'{path}/frame_{frame:04d}.jpg'
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        height_extension = 100
        new_height = img.shape[0] + height_extension
        new_image = np.zeros((new_height, img.shape[1]), dtype=np.float32)
        new_image[:img.shape[0], :] = img
        new_image = cv2.resize(new_image, (512, 512))
        new_image = (new_image / 255.0) * 2 - 1
        result = draw_img(new_image, smooth_kp[frame])
        cv2.imwrite(os.path.join(output_path, f'frame_{frame:04d}.jpg'), result)
    json_dict[path.split('/')[-1]] = str(smooth_kp.tolist())
    print(json_dict)


In [18]:
import os
import json

json_dict = {}
for path in os.listdir('../test_frames'):
    path = os.path.join('../test_frames', path)
    output_path = os.path.join('../output', path.split('/')[-1])
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    kp_array = draw_pred_img(path, model, seq_len=15)
    smooth_kp = apply_kalman_filter(kp_array)
    draw_smth_kp(path, output_path, smooth_kp, json_dict)

with open('../output/smoothed_kp.json', 'w') as f:
    json.dump(json_dict, f)



 62%|██████▏   | 1644/2650 [00:55<00:34, 29.42it/s]


KeyboardInterrupt: 